In [9]:
import numpy as np
import pandas as pd

Варіант 5

S = (F, 8, E, 9, 7, 2, 0, D, C, 6, 1, 5, B, 4, 3, A)

In [10]:
s_block = [0xF, 0x8, 0xE, 0x9, 0x7, 0x2, 0x0, 0xD, 0xC, 0x6, 0x1, 0x5, 0xB, 0x4, 0x3, 0xA]
s_block_inverse = [0x6, 0xA, 0x5, 0xE, 0xD, 0xB, 0x9, 0x4, 0x1, 0x3, 0xF, 0xC, 0x8, 0x7, 0x2, 0x0]
ENC = True
NOT_ENC = False

class heys:
    
    def heys_round(self, n, key = 0):
        ct = n ^ key
        ct = self.substitute(ct , s_block)
        ct = self.mix_words(ct)
        return ct

    # r = (s_1(y_1), s_2(y_2), ... , s_n(y_n)) 
    def substitute(self, n, enc):
        s = s_block if enc == True else s_block_inverse
        r = 0
        for i in range(4):
            r |= s[(n >> (i * 4)) & 15] << (i * 4)
        return r
        
    # i-тий біт j-того фрагменту стає j-тим бітом i-того фрагменту.
    def mix_words(self, n):
        r = 0
        for j in range(4):
            for i in range(4):
                r |= (n >> (4 * j + i) & 1) << (4 * i + j)
        return r
              

Створюємо таблицю диференціалів

$$ f(x \circ a) \bullet (f(x))^{-1} = b $$
так як у нас лише операції $\oplus$, тож воно претворюється на 
$$ f(x \oplus a) \oplus f(x) = b $$

In [11]:
heys_obj = heys()
# таблиця диференціалів для перестановки шифру Хейса
diff_substitution = np.zeros((16, 16))

for alpha in range(16):
    for beta in range(16):
        for x in range(16):
            diff_substitution[alpha][beta] += heys_obj.substitute(x ^ alpha, ENC) == heys_obj.substitute(x, ENC) ^ beta

diff_substitution /= 16

pd.DataFrame(diff_substitution)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.0,0.000,0.000,0.000,0.125,0.125,0.000,0.250,0.000,0.125,0.125,0.000,0.000,0.125,0.000,0.125
2,0.0,0.250,0.000,0.125,0.000,0.000,0.000,0.125,0.125,0.000,0.000,0.000,0.000,0.125,0.125,0.125
3,0.0,0.125,0.125,0.000,0.000,0.000,0.250,0.250,0.000,0.125,0.125,0.000,0.000,0.000,0.000,0.000
4,0.0,0.000,0.250,0.000,0.125,0.000,0.000,0.125,0.125,0.000,0.125,0.000,0.000,0.000,0.125,0.125
5,0.0,0.000,0.000,0.125,0.000,0.000,0.125,0.000,0.125,0.125,0.000,0.125,0.000,0.250,0.000,0.125
6,0.0,0.125,0.000,0.000,0.000,0.125,0.000,0.000,0.000,0.125,0.125,0.125,0.125,0.000,0.000,0.250
7,0.0,0.000,0.125,0.000,0.000,0.250,0.125,0.000,0.125,0.000,0.000,0.000,0.125,0.000,0.250,0.000
8,0.0,0.000,0.000,0.250,0.000,0.000,0.125,0.125,0.000,0.000,0.000,0.000,0.250,0.000,0.125,0.125
9,0.0,0.000,0.000,0.125,0.125,0.000,0.000,0.000,0.125,0.250,0.125,0.125,0.000,0.000,0.125,0.000


In [12]:
p = 0.00001

def differential_search(alpha):
    return 0

In [13]:
def extract_keys():
    return 0